# ***Lab 3: Text Preprocessing***

## **Exercise 1:** Lowercasing

In [1]:
# Example customer reviews
reviews = [
    "The Product Quality is Excellent!",
    "CUSTOMER Service was Terrible.",
    "Fast Delivery and Great Packaging."
]

# Apply lowercasing
lowercased_reviews = [review.lower() for review in reviews]

# Display results
for original, lowered in zip(reviews, lowercased_reviews):
    print(f"Original: {original}")
    print(f"Lowercased: {lowered}")
    print("-" * 50)

Original: The Product Quality is Excellent!
Lowercased: the product quality is excellent!
--------------------------------------------------
Original: CUSTOMER Service was Terrible.
Lowercased: customer service was terrible.
--------------------------------------------------
Original: Fast Delivery and Great Packaging.
Lowercased: fast delivery and great packaging.
--------------------------------------------------


## **Exercise 2:** Removing Punctuation and Special Characters

In [2]:
import re

# Example customer reviews
reviews = [
    "The product quality is excellent!!! 😍",
    "Customer service was terrible, not happy... :(",
    "Fast delivery & great packaging @GlobalMart",
    "Refund requested due to damaged item #12345."
]

# Function to remove punctuation and special characters
def remove_punctuations(text):
    # Keep only letters, numbers, and spaces
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Apply cleaning
cleaned_reviews = [remove_punctuations(review) for review in reviews]

# Display results
for original, cleaned in zip(reviews, cleaned_reviews):
    print(f"Original:   {original}")
    print(f"Cleaned:    {cleaned}")
    print("-" * 50)

Original:   The product quality is excellent!!! 😍
Cleaned:    The product quality is excellent 
--------------------------------------------------
Original:   Customer service was terrible, not happy... :(
Cleaned:    Customer service was terrible not happy 
--------------------------------------------------
Original:   Fast delivery & great packaging @GlobalMart
Cleaned:    Fast delivery  great packaging GlobalMart
--------------------------------------------------
Original:   Refund requested due to damaged item #12345.
Cleaned:    Refund requested due to damaged item 12345
--------------------------------------------------


## **Exercise 3:** Removing Stopwords

In [4]:
import nltk
from nltk.corpus import stopwords
import string

# Download stopwords (only needed once)
nltk.download('stopwords')

# Sample text
text = "This is an example sentence, showing how to remove stopwords and punctuation!"

# Convert text to lowercase
text = text.lower()

# Remove punctuation
text = text.translate(str.maketrans('', '', string.punctuation))

# Tokenize (split into words)
words = text.split()

# Get English stopwords
stop_words = set(stopwords.words('english'))

# Separate stopwords and non-stopwords
filtered_words = [word for word in words if word not in stop_words]
removed_stopwords = [word for word in words if word in stop_words]

print("Original words:", words)
print("Filtered words (stopwords removed):", filtered_words)
print("Stopwords removed:", removed_stopwords)


Original words: ['this', 'is', 'an', 'example', 'sentence', 'showing', 'how', 'to', 'remove', 'stopwords', 'and', 'punctuation']
Filtered words (stopwords removed): ['example', 'sentence', 'showing', 'remove', 'stopwords', 'punctuation']
Stopwords removed: ['this', 'is', 'an', 'how', 'to', 'and']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Exercise 4:** Stemming

In [6]:
import re
from nltk.stem import PorterStemmer

# Initialize stemmer
stemmer = PorterStemmer()

# Sample text
text = "The customers are running, studies show they were happier after receiving their products."

# Simple tokenization using regex (no nltk punkt needed)
words = re.findall(r"\b\w+\b", text.lower())  # extract words only

# Apply stemming
stemmed_words = [stemmer.stem(word) for word in words]

# Compare original vs stemmed
print("Original words:", words)
print("Stemmed words:", stemmed_words)


Original words: ['the', 'customers', 'are', 'running', 'studies', 'show', 'they', 'were', 'happier', 'after', 'receiving', 'their', 'products']
Stemmed words: ['the', 'custom', 'are', 'run', 'studi', 'show', 'they', 'were', 'happier', 'after', 'receiv', 'their', 'product']


## **Exercise 5:** Lemmatization

In [7]:
# Install spaCy model (only first time, then comment this out)
# !python -m spacy download en_core_web_sm

import spacy

# Load English model
nlp = spacy.load("en_core_web_sm")

# Sample text
text = "The customers are running, studies show they were happier after receiving their products."

# Process text with spaCy
doc = nlp(text)

# Extract original tokens and their lemmas
lemmas = [(token.text, token.lemma_) for token in doc if token.is_alpha]

# Print results
print("Original vs Lemmatized:")
for word, lemma in lemmas:
    print(f"{word:15} --> {lemma}")

Original vs Lemmatized:
The             --> the
customers       --> customer
are             --> be
running         --> run
studies         --> study
show            --> show
they            --> they
were            --> be
happier         --> happy
after           --> after
receiving       --> receive
their           --> their
products        --> product


## **Exercise 6:** Text Normalization

In [9]:
import re
import spacy
import string

# Load spaCy English model for lemmatization
nlp = spacy.load("en_core_web_sm")

# Business text example
text = """
Hi John, thanks for ordering from BusinessCorp!
Your order #12345 was placed on 12/09/2025 for $499.99.
Visit https://businesscorp.com/orders/12345 to track it.
If you have questions, email support@businesscorp.com or call +971-50-123-4567.
We’re offering 10% off your next order—don’t miss it!
"""

# Dictionary for contractions
contractions = {
    "don’t": "do not",
    "can't": "cannot",
    "won’t": "will not",
    "i'm": "i am",
    "we’re": "we are",
    "they’re": "they are",
    "it's": "it is",
    "you're": "you are",
}

def expand_contractions(text, contractions_dict):
    pattern = re.compile("|".join(re.escape(k) for k in contractions_dict.keys()), flags=re.IGNORECASE)
    return pattern.sub(lambda m: contractions_dict[m.group(0).lower()], text)

def normalize_text(text):
    # 1. Lowercasing
    text = text.lower()

    # 2. Expand contractions
    text = expand_contractions(text, contractions)

    # 3. Replace business entities
    text = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b", "<EMAIL>", text)   # Emails
    text = re.sub(r"http[s]?://\S+", "<URL>", text)                                          # URLs
    text = re.sub(r"\+?\d{1,3}[-\s]?\(?\d{1,4}\)?[-\s]?\d{3}[-\s]?\d{4}", "<PHONE>", text)   # Phone numbers
    text = re.sub(r"\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b", "<DATE>", text)                      # Dates
    text = re.sub(r"\$\d+(?:\.\d{2})?", "<MONEY>", text)                                     # Money
    text = re.sub(r"\d+(\.\d+)?%", "<PERCENT>", text)                                        # Percentages
    text = re.sub(r"\b\d+\b", "<NUM>", text)                                                 # Plain numbers

    # 4. Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # 5. Lemmatization
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc if token.is_alpha])

    return text

# Apply normalization
normalized = normalize_text(text)

print("Original Text:\n", text)
print("\nNormalized Text:\n", normalized)

Original Text:
 
Hi John, thanks for ordering from BusinessCorp! 
Your order #12345 was placed on 12/09/2025 for $499.99.
Visit https://businesscorp.com/orders/12345 to track it.
If you have questions, email support@businesscorp.com or call +971-50-123-4567.
We’re offering 10% off your next order—don’t miss it!


Normalized Text:
 hi john thank for order from businesscorp your order NUM be place on date for money visit url to track it if you have question email email or call phone we be offer PERCENT off your next order do not miss it


## **Exercise 7:** Handlings Numbers, Dates, URLs, Emails, etc.

In [8]:
import re

# Sample business text
text = """
Customer John.Doe@example.com placed an order on 12/09/2025 for $499.99.
Visit https://example.com/order/12345 to track it.
Contact us at support@business.com or +971-50-123-4567.
The delivery success rate was 98.5% in Q3.
"""

# Define regex patterns
patterns = {
    "EMAIL": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b",
    "URL": r"http[s]?://\S+",
    "DATE": r"\b(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4})\b",
    "PHONE": r"\+?\d{1,3}[-\s]?\(?\d{1,4}\)?[-\s]?\d{3}[-\s]?\d{4}",
    "MONEY": r"\$\d+(?:\.\d{2})?",
    "PERCENT": r"\d+(\.\d+)?%",
    "NUMBER": r"\b\d+\b"
}

# Function to normalize text
def normalize_text(text):
    for label, pattern in patterns.items():
        text = re.sub(pattern, f"<{label}>", text)
    return text

# Apply normalization
normalized_text = normalize_text(text)

print("Original Text:\n", text)
print("\nNormalized Text:\n", normalized_text)

Original Text:
 
Customer John.Doe@example.com placed an order on 12/09/2025 for $499.99.
Visit https://example.com/order/12345 to track it. 
Contact us at support@business.com or +971-50-123-4567.
The delivery success rate was 98.5% in Q3.


Normalized Text:
 
Customer <EMAIL> placed an order on <DATE> for <MONEY>.
Visit <URL> to track it. 
Contact us at <EMAIL> or <PHONE>.
The delivery success rate was <PERCENT> in Q3.

